In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import *
from pyspark import SparkConf
import pyspark as ps
import pandas as pd

In [2]:
conf = (SparkConf()
            .setAppName("TestRdd") \
            .set('spark.driver.cores', '1') \
            .set('spark.executor.cores', '3') \
            .set('spark.driver.memory', '4G') \
            .set('spark.executor.memory', '16G'))\
            .set("spark.network.timeout", "15000s")
sc = SparkContext('local[*]', conf = conf)
sqlContext = SQLContext(sc)

In [24]:
data_reviews =  sqlContext.read.parquet('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/review.parquet')
data_usuarios = sqlContext.read.csv('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/user.csv', sep=',', multiLine=True,header=True, inferSchema=True)

In [25]:
usuarios = data_usuarios.rdd
reviews = data_reviews.rdd

In [26]:
fecha_registro = usuarios.map(lambda x: pd.to_datetime(x.yelping_since).to_pydatetime())

In [27]:
cantidad_usuarios = usuarios.count()

In [28]:
import datetime
fecha_actual = datetime.datetime.now()
antiguedad_usuarios = fecha_registro.map(lambda x: (fecha_actual - x).days).reduce(lambda x,y: int(x)+int(y))                            

In [29]:
print("Antiguedad promedio de los usuarios: ", antiguedad_usuarios/cantidad_usuarios,"DIAS")
print("Antiguedad promedio de los usuarios: ", (antiguedad_usuarios/cantidad_usuarios)/365,"AÑOS")

Antiguedad promedio de los usuarios:  2780.86147672641 DIAS
Antiguedad promedio de los usuarios:  7.618798566373726 AÑOS


In [ ]:
#el nombre del usuario más antiguo cuyas última review contenga la palabra ‘pizza’

In [5]:
usuarios_id = usuarios.map(lambda x: (x.user_id,((fecha_actual - (pd.to_datetime(x.yelping_since).to_pydatetime())).days,x.name)))
reviews_text = reviews.map(lambda x: (x.user_id,(x.text,pd.to_datetime(x.date).to_pydatetime())))

In [18]:
pizza_reviews = reviews_text.reduceByKey(lambda x,y: x if x[1] > y[1] else y)\
                            .filter(lambda x: "pizza" in x[1][0])\
                            .map(lambda x: (x[0],x[1][1]))
#Me quedo con las reviews ultimas reviews de cada usuario y filtro las que contienen la palabra pizza

In [30]:
join_usuarios_reviews = pizza_reviews.join(usuarios_id) 

In [40]:
join_usuarios_reviews.map(lambda x: (x[1][1][1],x[1][1][0])).reduce(lambda x,y: x if x > y else y)
#Obtengo el usuario mas antiguo

('雨琪', 2062)

In [ ]:
#(nombre,antiguedad en dias)